# AWS CloudFront

<img src="../_assets/aws_service_icons/cloudfront.svg" width="80" alt="AWS CloudFront">

## Goals

- Understand what **CloudFront** is (at a high level)
- Know what it’s **practically used for**
- See a **simple SDK pseudo-code** workflow (no execution)

---

## What CloudFront is

**Amazon CloudFront** is AWS’s global **Content Delivery Network (CDN)**. You create a **distribution** that sits in front of an **origin** (for example: an S3 bucket, an Application Load Balancer, an API Gateway endpoint, or any HTTP server).

CloudFront routes users to nearby **edge locations**, serving content from cache when possible and fetching from the origin when needed. It can also terminate TLS, enforce security controls (WAF, geo restrictions), and run lightweight logic at the edge (CloudFront Functions / Lambda@Edge).

Key terms:

- **Distribution**: the CloudFront configuration + the `dxxxxx.cloudfront.net` hostname (and optional custom domain)
- **Origin**: where CloudFront fetches content from when it’s not cached
- **Cache behavior**: rules that decide *what* is cached and *how* requests are forwarded (paths, methods, headers, cookies, query strings)
- **Invalidation**: a request to remove cached objects early (before TTL expiry)

---

## What it’s practically used for

Common real-world uses:

- **Static website / SPA delivery**: cache HTML/CSS/JS/images close to users (often with S3 as the origin)
- **API acceleration**: put CloudFront in front of an API to reduce latency and offload TLS (optionally caching selected GET responses)
- **Large file downloads**: faster, more reliable delivery (and reduced origin bandwidth)
- **Media streaming**: video-on-demand and live streaming patterns
- **Origin protection**: restrict direct access to origins (for S3, typically via Origin Access Control)
- **Security controls at the edge**: AWS WAF, rate limiting patterns, geo restrictions, signed URLs/cookies

When CloudFront helps most:

- your users are geographically distributed
- responses are cacheable (static assets, versioned files, infrequently changing content)
- you want to reduce load and egress from your origin

---

## SDK pseudo-code (do not run)

Below is a **conceptual** example using the AWS SDK for Python (**boto3**). The real `DistributionConfig` schema is verbose; this is intentionally simplified to illustrate the workflow.

```python
# PSEUDO-CODE (do not execute)

import boto3
from time import time

cf = boto3.client('cloudfront')

# 1) Create a CloudFront distribution in front of an S3 origin
resp = cf.create_distribution(
    DistributionConfig={
        'CallerReference': str(time()),
        'Comment': 'CloudFront in front of S3 (example)',
        'Enabled': True,
        'Origins': {
            'Quantity': 1,
            'Items': [
                {
                    'Id': 's3-origin',
                    'DomainName': 'my-bucket.s3.amazonaws.com',
                    'S3OriginConfig': {
                        # In practice: use Origin Access Control (OAC) / bucket policies so the bucket isn’t public
                    },
                }
            ],
        },
        'DefaultCacheBehavior': {
            'TargetOriginId': 's3-origin',
            'ViewerProtocolPolicy': 'redirect-to-https',
            # In practice: attach CachePolicyId / OriginRequestPolicyId instead of forwarding everything
        },
    },
)

distribution_id = resp['Distribution']['Id']
domain_name = resp['Distribution']['DomainName']  # e.g. d111111abcdef8.cloudfront.net

# 2) Later, when content changes, invalidate cached paths (use sparingly)
cf.create_invalidation(
    DistributionId=distribution_id,
    InvalidationBatch={
        'CallerReference': str(time()),
        'Paths': {'Quantity': 1, 'Items': ['/index.html']},
    },
)

# 3) (Optional) Wait until the distribution is fully deployed before switching DNS
# cf.get_waiter('distribution_deployed').wait(Id=distribution_id)

print('Use CloudFront URL:', f'https://{domain_name}')
```

Notes:

- For production, CloudFront is usually managed with **IaC** (CloudFormation / CDK / Terraform) to keep the config versioned and repeatable.
- Cache correctness depends on **Cache-Control** headers, TTLs, and which request parts (headers/cookies/query strings) are included in the cache key.

---

## References

- AWS CloudFront documentation
- boto3 CloudFront client reference
